In [17]:
import datetime as dt
import functools
import multiprocessing
import time
from enum import Enum

import plotly.graph_objs as go
import plotly.io as po
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

pd.set_option('display.expand_frame_repr', False)
pd.options.mode.chained_assignment = None


In [18]:
def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        function_info = f"  ==> Finished {func.__name__!r} in".ljust(50)
        print(f"{function_info} {run_time:.4f} secs")
        return value
    return wrapper_timer
allo = 'allo'

In [19]:
def get_cached_data_acs(fermenter, days, q):
    data_template = 'cache_data/brewery_data_ferm_{fermenter}_day_{day}.csv'
    data_files = [data_template.format(fermenter=str(fermenter), day=str(d)) for d in range(days)]
    df_list = []
    for day, filename in enumerate(data_files):
        with open(filename) as f:
            df_list.append(pd.read_csv(f))
    df = pd.concat(df_list, sort=True)
    q.put(df)


@timer
def all_brand_data_read(num_days):
    t0 = dt.datetime.now()
    print('Step 1 (start): read data from local cache... ', end='', flush=True)
    q = multiprocessing.Queue()
    worker = []
    for fermenter in range(31, 37):
        t = multiprocessing.Process(target=get_cached_data_acs, args=(fermenter, num_days, q))
        worker.append(t)
        worker[-1].start()

    df = pd.DataFrame()
    for _ in range(31, 37):
        df_temp = q.get()
        df = df.append(df_temp, ignore_index=True)

    for t in worker:
        t.join()

    print('done, time:', dt.datetime.now() - t0)
    df.to_csv(f'beer_case2_{num_days}.csv')
    return df


@timer
def all_brand_data_csv(num_days):
    return pd.read_csv(f'beer_case2_{num_days}.csv')

In [20]:
class Pos(Enum):
    bottom = 1
    middle = 2
    top = 3


pressure_valve = {p: p.name.capitalize() + ' TIC PV' for p in Pos}
BOTTOM_PV = pressure_valve[Pos.bottom]
MIDDLE_PV = pressure_valve[Pos.middle]
TOP_PV = pressure_valve[Pos.top]
TIC_PVS = [BOTTOM_PV, TOP_PV, MIDDLE_PV]

BOTTOM_OUT = BOTTOM_PV.replace('PV', 'OUT')
MIDDLE_OUT = MIDDLE_PV.replace('PV', 'OUT')
TOP_OUT = TOP_PV.replace('PV', 'OUT')
TIC_OUTS = [BOTTOM_OUT, TOP_OUT, MIDDLE_OUT]

BAD_INPUT = 'Bad Input'
COMM_FAILURES = ['I/O Timeout', 'Comm Fail']
POST_FERMENTATION_STAGES = ['Fermentation', 'Free Rise', 'Diacetyl Rest', 'Cooling']

In [21]:
@timer
def brand_df_cleanup(brand_df):
    # Remove all data point with bad input
    brand_df = brand_df \
        .drop(brand_df[brand_df.Brand == BAD_INPUT].index) \
        .drop(brand_df[brand_df.Status == BAD_INPUT].index) \
        .drop(brand_df[brand_df[TOP_PV] == BAD_INPUT].index |
              brand_df[brand_df[MIDDLE_PV] == BAD_INPUT].index |
              brand_df[brand_df[TOP_PV] == BAD_INPUT].index,
              errors='ignore')

    # Keep only fermentation or post-fermentation stages
    brand_status_df = brand_df.loc[brand_df['Status'].isin(POST_FERMENTATION_STAGES)]

    # Remove all data points with communication issues
    for tic_pv in TIC_PVS:
        brand_status_df = brand_status_df[~(brand_status_df[tic_pv].isin(COMM_FAILURES))]
        brand_status_df[tic_pv] = brand_status_df[tic_pv].astype(float)

    return brand_df, brand_status_df

In [22]:
@timer
def cooling_data_extraction(fermentation_df, brand_status_df, brand, use_temp_position):
    # Provides the corrected time offset post fermentation
    brand_status_df = fermentation_times(brand_status_df, fermentation_df, brand)

    for tic_out in TIC_OUTS:
        brand_status_df[tic_out] = pd.to_numeric(brand_status_df[tic_out], errors='coerce')

    # condition for it to be in cooling phase
    cool_stage = brand_status_df[
        (brand_status_df[TOP_OUT] > 99.99) &
        (brand_status_df[MIDDLE_OUT] > 99.99) &
        (brand_status_df[BOTTOM_OUT] > 99.99)]

    # get the first cooling step for each fermentation stage
    cooling_start_frame = cool_stage.groupby('label').first().reset_index()

    cooling_data = []
    for position in use_temp_position:
        if position.value:
            cooling_data.append(get_cooling_frames(cool_stage, cooling_start_frame, brand, position))

    return cooling_data


@timer
def get_cooling_frames(cool_stage, cooling_start_frame, brand, position):  # bottom=False, middle=True, top=False):
    start_time = 0
    end_time = 3.5  # in days, longest cooling period possible

    cooling_column = 'Time since cooling'
    cool_stage.loc[:, 'Time since cooling'] = -1
    cooling_stage = pd.DataFrame()
    if len(cooling_start_frame) > 0:
        for index, row in cooling_start_frame.iterrows():
            label = row['label2']  # get the unique label
            cool_start_time = row['tsf3']  # get the unique start of cooling to each label
            # Each unique label is associated with a fermentation stage for a brand
            mask = (cool_stage['label2'] == label)  # get those rows with that same label
            cool_stage_valid = cool_stage[mask]

            tic_pv = pressure_valve[position]
            # get only frames that have the bottom process variable greater than 50
            if float(row[tic_pv]) > 50:  # and keep [CF]
                # subtract the start of cooling from each individual cooling step
                cool_stage.loc[mask, cooling_column] = cool_stage_valid['tsf3'] - cool_start_time

                cool_stage_current = cool_stage[(cool_stage[cooling_column] >= start_time) &
                                                (cool_stage[cooling_column] < end_time)]
                # make sure the labels are all positive, make sure these are post fermentation stages
                cool_stage_current = cool_stage_current[cool_stage_current['label'] >= 0]
                # get only the max of the post fermentation stages
                cool_stage_current[tic_pv] = cool_stage_current.groupby([cooling_column])[tic_pv].transform(max)
                cool_stage_current = cool_stage_current.rename(columns={tic_pv: 'temperature', cooling_column: 'tsc'})
                cool_stage_current = cool_stage_current[['temperature', 'tsc', 'Brand', 'label', 'Volume']]
                cooling_stage = cool_stage_current
    else:
        print("!!! Sorry no cooling stage associated with " + str(brand) + " brand")

    return cooling_stage

In [23]:
# Return the rows when fermentation start for a brand
@timer
def fermentation_starts_naive(brand_frame, all_frame):
    fermentation_start_list = []
    for i, row in brand_frame.iterrows():
        if row['Status'] == 'Fermentation' and all_frame.loc[i - 1]['Status'] != 'Fermentation':
            fermentation_start_list.append(row)
    return fermentation_start_list


# Return the rows when fermentation start for a brand
@timer
def fermentation_starts_optimized(brand_df):
    df = brand_df[(brand_df['Status'] == 'Fermentation') & (brand_df['Status'].shift(1) != 'Fermentation')]
    fermentation_starts = [row for _, row in df.iterrows()]
    return fermentation_starts


# Get the time since fermentation
@timer
def fermentation_times(brand_frame, fermentation_frames, brand):
    brand_frame['tsf2'] = 100000  # initializing the temp variables
    brand_frame['tsf3'] = 100000  # init the temp variables
    brand_frame['label'] = -1  # label is to label all fermentation processes
    if brand == 'Realtime Hops':
        print(len(fermentation_frames))
    count = 0
    for index, fermentation_frame in enumerate(fermentation_frames):
        fermentation_time = fermentation_frame['Timestamp']
        brand_frame['label'] = brand_frame['Timestamp'].apply(
            lambda x: count if pd.Timestamp(x) >= pd.Timestamp(fermentation_time) else -1)
        brand_frame['tsf2'] = brand_frame['Timestamp'].apply(
            lambda x: ((pd.Timestamp(x)) - (pd.Timestamp(fermentation_time))).total_seconds() if pd.Timestamp(
                x) >= pd.Timestamp(fermentation_time) else 1000000000)
        brand_frame['tsf2'] = brand_frame['tsf2'].apply(lambda x: x / 86400)  # convert time to days
        if count > 0:
            # the min of the two is the actual time since fermentation start
            brand_frame['tsf2'] = brand_frame[['tsf2', 'tsf3']].min(axis=1)
            mask = (brand_frame['label'] == -1)
            brand_frame_valid = brand_frame[mask]
            brand_frame.loc[mask, 'label'] = brand_frame_valid['label2']

        brand_frame['tsf3'] = brand_frame['tsf2']
        brand_frame['label2'] = brand_frame['label']
        count += 1

    # if there is any zero just remove that
    brand_frame = brand_frame[(brand_frame['tsf3'] <= 100000) & (brand_frame['label2'] >= 0)]

    return brand_frame

In [29]:
def temperature_profile(x, a, b):
    # Unpack x values
    temperature = x[0]
    volume = x[1]
    return temperature + np.multiply(temperature, np.multiply(a, np.reciprocal(volume))) - a * b * np.reciprocal(volume)

In [38]:
# %%debug
# import pdb
# from pdb import set_trace as bp
def main(brand, temp_sensors, training_days, testing_days):
    """
    Input parameters:
    * brand to consider
    * temperature sensor position to use for computation
    * number of days to compute prediction parameters
    * number of days for testing
    """
    # ['5450' 'Bad Input' 5450 nan 'Alistair' 'Kerberos' 'Realtime Hops'
    #  'Red Wonder' 'Grey Horse']
    # brand = 'Kerberos'
    use_temp_position = {
        Pos.bottom: temp_sensors['bottom'],
        Pos.middle: temp_sensors['middle'],
        Pos.top: temp_sensors['top']
    }
    # training_days = 20
    # testing_days = 200
    optimized_fermentation_start = True
    num_day = [training_days]  # testing_days]
    t00 = dt.datetime.now()
    for num_days in num_day:
        all_df = all_brand_data_read(num_days)
        # all_df = all_brand_data_csv(num_days)
        brand_df = all_df[all_df.Brand == brand]
        brand_df, brand_status_df = brand_df_cleanup(brand_df)
        # Get fermentation starts: each brand has particular rows that signify the start of fermentation
        if not optimized_fermentation_start:
            fermentation_df = fermentation_starts_naive(brand_df, all_df)
        else:
            fermentation_df = fermentation_starts_optimized(brand_df)
        if len(fermentation_df) == 0:
            print('!!! No fermentation data for brand:', brand)
            return 1
        cooling_data = cooling_data_extraction(fermentation_df, brand_status_df, brand, use_temp_position)

        if len(cooling_data) > 0:
            # bp()
            # Concatenating the cooling_data list into a single dataframe
            cool_df = pd.concat(cooling_data)
            # sort the temperatures in a descending fashion
            cool_df = cool_df.sort_values(by=['temperature'], ascending=False)
            cool_df.to_csv(f'cool_df_{training_days}days.csv')

            # get the y value for the x, this will be used in curve fitting
            cool_df['temp_y'] = cool_df['temperature'].shift(-1)
            cool_df = cool_df[:-1]  # drop the last row

            if num_days == training_days:

                cool_df_training = pd.DataFrame()
                lbl = 0

                while cool_df_training.empty:
                    cool_df_training = cool_df[cool_df.label == lbl]
                    lbl += 1
                cool_df_training.to_csv(f'cool_df_training{training_days}days.csv')
                x1_train = cool_df_training.temperature.values  # training temperature feature
                x2_train = cool_df_training.Volume.values.astype(float)  # training Volume feature
                x = [x1_train, x2_train]  # [temperature, volume]

                # Training of non-linear least squares model
                # Nonlinear curve-fitting pass a tuple in curve fitting
                popt, pcov = curve_fit(temperature_profile, x, cool_df_training.temp_y.values)
                a = popt[0]  # get the coefficient a in the model
                b = popt[1]  # get the coefficient b in the model

                # Get the initial point of all temperature curves
                # y_first = [x1_train[0] + i for i in range(-8, 9, 4)]  # plot on either side of the initial temperature
                y_first = [x1_train[0]]  # if you want to plot a single data field

                # Compute the prediction for each individual start temperature
                for y_predicted in y_first:
                    y_pred = [y_predicted]
                    label = 'Prediction curve, start temp: {:5.2f}F'.format(y_predicted)
                    cool_df_training = cool_df_training.sort_values(by=['tsc'])
                    for i in range(1, len(x2_train)):
                        y_predicted = y_predicted * (1 + (a / x2_train[i])) - (a * b / x2_train[i])
                        y_pred.append(y_predicted)
                    data = [go.Scatter(x = cool_df_training.tsc.values, y = y_pred, mode='lines', name=label, marker=dict(color='blue'))]
                    # plt.plot(cool_df_training.tsc.values, y_pred, label=label, color='blue')

            t01 = dt.datetime.now()
            print('Time taken:', t01 - t00)
            # Plotting the actual data and other plot configurations
            if num_days == training_days:
                data.append(go.Scatter(x = cool_df.tsc.values, 
                                        y = cool_df.temperature.values, 
                                        mode='markers', 
                                        name='Actual Data', 
                                        opacity=0.4,
                                        marker=dict(color='orange')))
                # plt.scatter(cool_df.tsc.values, cool_df.temperature.values, label='Actual data', color='m')
                # plt.xlabel('Time (days)')
                # plt.ylabel('Temperature (F)')
                # plt.title(str(brand) + ': cooling pred - curve -' + str(num_days) + " days")
                # plt.legend()

            # Save the figure
            # output_dir = '.'
            # if len(y_first) > 1:
            #     location = '{}/' + str(brand) + '_prediction_constant_volume' + '.png'
            # else:
            #     location = '{}/' + str(brand) + '_prediction_ATT' + '.png'
            # plt.savefig(location.format(output_dir))
    # plt.show()
    # fig = go.FigureWidget(data=trace)
    return data, len(fermentation_df)

In [42]:
# Tenant is osisoft-academia
tenant_id = '4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93'
client_id = 'ff8220f7-6b7c-4477-b21e-8e2ca20649d4'  
client_secret = 'tRiVPtWc6kgcxEw090Qi/7nwA+JfI4cLlaL34Edgx+M='
brand = 'Kerberos'
# Temperature sensor position to consider
temp_sensors = {'bottom': False, 'middle': True, 'top': False}
training_days = 30
testing_days = 200

In [43]:
data, num_fermentation = main(brand, temp_sensors, training_days, testing_days)

Step 1 (start): read data from local cache... done, time: 0:00:03.668079
  ==> Finished 'all_brand_data_read' in            7.7016 secs
  ==> Finished 'brand_df_cleanup' in               0.1023 secs
  ==> Finished 'fermentation_starts_optimized' in  0.0097 secs
  ==> Finished 'fermentation_times' in             3.4317 secs
  ==> Finished 'get_cooling_frames' in             0.0502 secs
  ==> Finished 'get_cooling_frames' in             0.0423 secs
  ==> Finished 'get_cooling_frames' in             0.0489 secs
  ==> Finished 'cooling_data_extraction' in        3.6283 secs


ValueError: operands could not be broadcast together with shapes (20960,) (10481,) 

In [41]:
layout = go.Layout(
    xaxis=dict(title='Cooling time (days)'), 
    yaxis=dict(title='Temperature (F)'), 
    title=f'Cooling of {brand} beer, {training_days} days of data, {num_fermentation} fermentation(s)'
)
fig = go.FigureWidget(data=data, layout=layout)
po.write_image(fig, f'kerberos_{training_days}days_{num_fermentation}ferms_cool_df.png')
fig

FigureWidget({
    'data': [{'marker': {'color': 'blue'},
              'mode': 'lines',
              'name':…